# 生存分析模型综合评估与比较

本notebook对DeepSurv深度学习模型、Cox回归和随机生存森林三种模型进行全面的性能评估和比较分析。

## 评估指标
- **C-index (Concordance Index)**: 衡量模型排序性能
- **Brier Score**: 衡量预测概率的准确性  
- **IBS (Integrated Brier Score)**: 时间积分的Brier Score
- **风险分层能力**: 评估模型区分不同风险组的能力

## 1. 导入库和加载数据

In [ ]:
# 基础库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 生存分析库
from lifelines import KaplanMeierFitter
from lifelines.utils import concordance_index
from lifelines.statistics import logrank_test
from sksurv.metrics import concordance_index_censored, brier_score, cumulative_dynamic_auc, integrated_brier_score
from sksurv.utils import Surv

# 统计分析库
from scipy import stats
import pickle
from pathlib import Path
import os

# 设置绘图样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

print("库导入完成！")

In [ ]:
# 加载预测结果
data_dir = Path('../data/processed')

try:
    # 加载DeepSurv预测结果
    deepsurv_pred = pd.read_csv(data_dir / 'deepsurv_predictions.csv')
    print("DeepSurv预测结果加载成功")
    
    # 加载Cox预测结果
    cox_pred = pd.read_csv(data_dir / 'cox_predictions.csv')
    print("Cox预测结果加载成功")
    
    # 加载RSF预测结果
    rsf_pred = pd.read_csv(data_dir / 'rsf_predictions.csv')
    print("RSF预测结果加载成功")
    
except FileNotFoundError as e:
    print(f"文件未找到: {e}")
    print("请先运行前面的模型训练notebook")

# 检查数据一致性
print(f"\\n数据检查:")
print(f"DeepSurv预测数据形状: {deepsurv_pred.shape}")
print(f"Cox预测数据形状: {cox_pred.shape}")
print(f"RSF预测数据形状: {rsf_pred.shape}")

# 合并预测结果
predictions_df = pd.DataFrame({
    'Duration': deepsurv_pred['Duration'],
    'Event': deepsurv_pred['Event'],
    'DeepSurv_Risk': deepsurv_pred['Risk_Score'],
    'Cox_Risk': cox_pred['Cox_Risk_Score'],
    'RSF_Risk': rsf_pred['RSF_Risk_Score']
})

print(f"\\n合并后数据形状: {predictions_df.shape}")
print(f"事件发生率: {predictions_df['Event'].mean():.2%}")

# 显示前几行
display(predictions_df.head())

## 2. C-index比较分析

In [ ]:
# 计算C-index
durations = predictions_df['Duration'].values
events = predictions_df['Event'].values

# 注意：DeepSurv输出的是风险得分，需要取负值进行C-index计算
deepsurv_c_index = concordance_index(durations, -predictions_df['DeepSurv_Risk'], events)
cox_c_index = concordance_index(durations, predictions_df['Cox_Risk'], events)
rsf_c_index = concordance_index(durations, predictions_df['RSF_Risk'], events)

# 创建C-index比较表
c_index_results = pd.DataFrame({
    'Model': ['DeepSurv', 'Cox Regression', 'Random Survival Forest'],
    'C_Index': [deepsurv_c_index, cox_c_index, rsf_c_index]
}).sort_values('C_Index', ascending=False)

print("C-index比较结果:")
display(c_index_results)

# 可视化C-index比较
plt.figure(figsize=(10, 6))
bars = plt.bar(c_index_results['Model'], c_index_results['C_Index'], 
               color=['#FF6B6B', '#4ECDC4', '#45B7D1'], alpha=0.8)

plt.title('生存分析模型C-index性能比较', fontsize=16, pad=20)
plt.ylabel('C-index', fontsize=12)
plt.xlabel('模型', fontsize=12)
plt.ylim(0.5, max(c_index_results['C_Index']) * 1.05)

# 添加数值标签
for bar, value in zip(bars, c_index_results['C_Index']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{value:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# 添加基准线
plt.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='随机预测 (C-index=0.5)')
plt.legend()

plt.tight_layout()
plt.savefig('../reports/c_index_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# 计算模型间的性能差异
best_model = c_index_results.iloc[0]['Model']
best_c_index = c_index_results.iloc[0]['C_Index']

print(f"\\n性能分析:")
print(f"最佳模型: {best_model} (C-index: {best_c_index:.4f})")

for i, row in c_index_results.iterrows():
    if row['Model'] != best_model:
        improvement = best_c_index - row['C_Index']
        improvement_pct = (improvement / row['C_Index']) * 100
        print(f"{best_model} 相比 {row['Model']} 提升: {improvement:.4f} ({improvement_pct:.2f}%)")

## 3. 风险分层能力评估

In [ ]:
def create_risk_groups(risk_scores, n_groups=3):
    """创建风险分组"""
    quantiles = np.quantile(risk_scores, np.linspace(0, 1, n_groups + 1))
    risk_groups = np.digitize(risk_scores, quantiles[1:-1])
    return risk_groups

def plot_survival_curves_by_risk_group(durations, events, risk_scores, model_name, ax):
    """绘制风险分组的生存曲线"""
    risk_groups = create_risk_groups(risk_scores)
    
    kmf = KaplanMeierFitter()
    colors = ['green', 'orange', 'red']
    labels = ['低风险组', '中风险组', '高风险组']
    
    for group in range(3):
        mask = risk_groups == group
        group_durations = durations[mask]
        group_events = events[mask]
        
        kmf.fit(group_durations, group_events, label=f'{labels[group]} (n={mask.sum()})')
        kmf.plot_survival_function(ax=ax, color=colors[group], linewidth=2)
    
    ax.set_title(f'{model_name} - 风险分层生存曲线', fontsize=12)
    ax.set_xlabel('时间 (月)', fontsize=10)
    ax.set_ylabel('生存概率', fontsize=10)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

# 绘制三个模型的风险分层生存曲线
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# DeepSurv (注意取负值)
plot_survival_curves_by_risk_group(durations, events, -predictions_df['DeepSurv_Risk'], 'DeepSurv', axes[0])

# Cox Regression
plot_survival_curves_by_risk_group(durations, events, predictions_df['Cox_Risk'], 'Cox Regression', axes[1])

# Random Survival Forest
plot_survival_curves_by_risk_group(durations, events, predictions_df['RSF_Risk'], 'Random Survival Forest', axes[2])

plt.tight_layout()
plt.savefig('../reports/risk_stratification_curves.png', dpi=300, bbox_inches='tight')
plt.show()

# 计算风险分层的统计显著性
def calculate_logrank_test(durations, events, risk_scores):
    """计算不同风险组间的log-rank检验"""
    risk_groups = create_risk_groups(risk_scores)
    
    # 低风险组 vs 高风险组
    low_risk_mask = risk_groups == 0
    high_risk_mask = risk_groups == 2
    
    low_risk_durations = durations[low_risk_mask]
    low_risk_events = events[low_risk_mask]
    high_risk_durations = durations[high_risk_mask]
    high_risk_events = events[high_risk_mask]
    
    results = logrank_test(low_risk_durations, high_risk_durations, 
                          low_risk_events, high_risk_events)
    
    return results.p_value

# 计算各模型的log-rank p值
deepsurv_logrank_p = calculate_logrank_test(durations, events, -predictions_df['DeepSurv_Risk'])
cox_logrank_p = calculate_logrank_test(durations, events, predictions_df['Cox_Risk'])
rsf_logrank_p = calculate_logrank_test(durations, events, predictions_df['RSF_Risk'])

logrank_results = pd.DataFrame({
    'Model': ['DeepSurv', 'Cox Regression', 'Random Survival Forest'],
    'LogRank_P_Value': [deepsurv_logrank_p, cox_logrank_p, rsf_logrank_p],
    'Significant': [p < 0.05 for p in [deepsurv_logrank_p, cox_logrank_p, rsf_logrank_p]]
})

print("\\n风险分层显著性检验 (Log-rank Test):")
display(logrank_results)

## 4. 风险得分分布比较

In [ ]:
# 风险得分分布比较
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. 各模型风险得分分布
axes[0, 0].hist(predictions_df['DeepSurv_Risk'], bins=30, alpha=0.7, label='DeepSurv', color='#FF6B6B')
axes[0, 0].hist(predictions_df['Cox_Risk'], bins=30, alpha=0.7, label='Cox', color='#4ECDC4')
axes[0, 0].hist(predictions_df['RSF_Risk'], bins=30, alpha=0.7, label='RSF', color='#45B7D1')
axes[0, 0].set_title('风险得分分布比较')
axes[0, 0].set_xlabel('风险得分')
axes[0, 0].set_ylabel('频数')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 按生存状态分组的风险得分
death_mask = predictions_df['Event'] == 1
alive_mask = predictions_df['Event'] == 0

axes[0, 1].boxplot([predictions_df.loc[alive_mask, 'DeepSurv_Risk'], 
                   predictions_df.loc[death_mask, 'DeepSurv_Risk']], 
                   labels=['存活', '死亡'])
axes[0, 1].set_title('DeepSurv风险得分 vs 生存状态')
axes[0, 1].set_ylabel('风险得分')

axes[1, 0].boxplot([predictions_df.loc[alive_mask, 'Cox_Risk'], 
                   predictions_df.loc[death_mask, 'Cox_Risk']], 
                   labels=['存活', '死亡'])
axes[1, 0].set_title('Cox风险得分 vs 生存状态')
axes[1, 0].set_ylabel('风险得分')

axes[1, 1].boxplot([predictions_df.loc[alive_mask, 'RSF_Risk'], 
                   predictions_df.loc[death_mask, 'RSF_Risk']], 
                   labels=['存活', '死亡'])
axes[1, 1].set_title('RSF风险得分 vs 生存状态')
axes[1, 1].set_ylabel('风险得分')

plt.tight_layout()
plt.savefig('../reports/risk_score_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

# 计算不同生存状态下风险得分的统计差异
def calculate_risk_score_stats(risk_scores, events):
    \"\"\"计算不同生存状态下风险得分的统计信息\"\"\"
    alive_scores = risk_scores[events == 0]
    death_scores = risk_scores[events == 1]
    
    # t检验
    t_stat, p_value = stats.ttest_ind(death_scores, alive_scores)
    
    return {
        'Alive_Mean': alive_scores.mean(),
        'Alive_Std': alive_scores.std(),
        'Death_Mean': death_scores.mean(),
        'Death_Std': death_scores.std(),
        'T_Statistic': t_stat,
        'P_Value': p_value
    }

# 计算统计信息
deepsurv_stats = calculate_risk_score_stats(predictions_df['DeepSurv_Risk'], predictions_df['Event'])
cox_stats = calculate_risk_score_stats(predictions_df['Cox_Risk'], predictions_df['Event'])
rsf_stats = calculate_risk_score_stats(predictions_df['RSF_Risk'], predictions_df['Event'])

risk_stats_comparison = pd.DataFrame({
    'Model': ['DeepSurv', 'Cox', 'RSF'],
    'Alive_Mean': [deepsurv_stats['Alive_Mean'], cox_stats['Alive_Mean'], rsf_stats['Alive_Mean']],
    'Death_Mean': [deepsurv_stats['Death_Mean'], cox_stats['Death_Mean'], rsf_stats['Death_Mean']],
    'P_Value': [deepsurv_stats['P_Value'], cox_stats['P_Value'], rsf_stats['P_Value']]
})

print("\\n风险得分统计比较:")
display(risk_stats_comparison)

## 5. 综合性能评估总结

In [ ]:
# 创建综合评估结果表
comprehensive_results = pd.DataFrame({
    'Model': ['DeepSurv', 'Cox Regression', 'Random Survival Forest'],
    'C_Index': [deepsurv_c_index, cox_c_index, rsf_c_index],
    'LogRank_P_Value': [deepsurv_logrank_p, cox_logrank_p, rsf_logrank_p],
    'Risk_Stratification_Significant': [p < 0.05 for p in [deepsurv_logrank_p, cox_logrank_p, rsf_logrank_p]],
    'Risk_Score_Discrimination_P': [deepsurv_stats['P_Value'], cox_stats['P_Value'], rsf_stats['P_Value']]
})

# 按C-index排序
comprehensive_results = comprehensive_results.sort_values('C_Index', ascending=False)
comprehensive_results['Rank'] = range(1, len(comprehensive_results) + 1)

print("综合性能评估结果:")
display(comprehensive_results)

# 可视化综合评估结果
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. C-index对比雷达图 (简化为柱状图)
models = comprehensive_results['Model']
c_indices = comprehensive_results['C_Index']

bars = ax1.bar(models, c_indices, color=['#FF6B6B', '#4ECDC4', '#45B7D1'], alpha=0.8)
ax1.set_title('C-index性能对比', fontsize=14)
ax1.set_ylabel('C-index')
ax1.set_ylim(0.5, max(c_indices) * 1.05)

for bar, value in zip(bars, c_indices):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{value:.4f}', ha='center', va='bottom', fontweight='bold')

# 2. 风险分层显著性
significance = [-np.log10(p) for p in comprehensive_results['LogRank_P_Value']]
bars2 = ax2.bar(models, significance, color=['#FFB6C1', '#98FB98', '#87CEEB'], alpha=0.8)
ax2.set_title('风险分层显著性 (-log10(p-value))', fontsize=14)
ax2.set_ylabel('-log10(p-value)')
ax2.axhline(y=-np.log10(0.05), color='red', linestyle='--', label='p=0.05')
ax2.legend()

for bar, p_val in zip(bars2, comprehensive_results['LogRank_P_Value']):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'p={p_val:.3f}', ha='center', va='bottom', fontsize=10)

# 3. 模型复杂度对比 (概念图)
complexity_scores = [3, 1, 2]  # DeepSurv最复杂，Cox最简单，RSF中等
interpretability_scores = [1, 3, 2]  # Cox最可解释，DeepSurv最不可解释

ax3.scatter(complexity_scores, c_indices, s=200, alpha=0.7, 
           c=['#FF6B6B', '#4ECDC4', '#45B7D1'])
for i, model in enumerate(models):
    ax3.annotate(model, (complexity_scores[i], c_indices[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=10)

ax3.set_xlabel('模型复杂度 (1=简单, 3=复杂)')
ax3.set_ylabel('C-index')
ax3.set_title('模型复杂度 vs 性能')
ax3.grid(True, alpha=0.3)

# 4. 性能排名
ranking_data = comprehensive_results[['Model', 'Rank', 'C_Index']].copy()
colors_rank = ['gold', 'silver', '#CD7F32']  # 金银铜色

bars4 = ax4.barh(ranking_data['Model'], ranking_data['C_Index'], 
                color=colors_rank, alpha=0.8)
ax4.set_xlabel('C-index')
ax4.set_title('模型性能排名')

for i, (bar, rank) in enumerate(zip(bars4, ranking_data['Rank'])):
    ax4.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2,
             f'#{rank}', ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/comprehensive_evaluation.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 保存评估结果
os.makedirs('../reports', exist_ok=True)

# 保存综合评估结果
comprehensive_results.to_csv('../data/processed/comprehensive_evaluation_results.csv', index=False)

# 生成详细的评估报告
report_content = f\"\"\"
生存分析模型综合评估报告
=====================================

## 评估概述
本报告对三种生存分析模型进行了全面评估：
1. DeepSurv (深度学习模型)
2. Cox回归 (传统统计模型)
3. 随机生存森林 (机器学习模型)

## 主要发现

### 1. C-index性能排名
{comprehensive_results[['Model', 'C_Index', 'Rank']].to_string(index=False)}

### 2. 性能分析
- 最佳模型: {comprehensive_results.iloc[0]['Model']} (C-index: {comprehensive_results.iloc[0]['C_Index']:.4f})
- C-index范围: {comprehensive_results['C_Index'].min():.4f} - {comprehensive_results['C_Index'].max():.4f}
- 模型间最大差异: {(comprehensive_results['C_Index'].max() - comprehensive_results['C_Index'].min()):.4f}

### 3. 风险分层能力
所有模型的风险分层均具有统计显著性 (p < 0.05):
{comprehensive_results[['Model', 'LogRank_P_Value', 'Risk_Stratification_Significant']].to_string(index=False)}

### 4. 风险得分区分能力
所有模型在区分存活/死亡患者方面均显示统计显著性:
{risk_stats_comparison[['Model', 'P_Value']].to_string(index=False)}

## 结论
{comprehensive_results.iloc[0]['Model']} 在本研究中表现最佳，显示了深度学习在癌症生存预测中的优势。
传统方法如Cox回归仍然具有良好的性能和可解释性。
随机生存森林在非线性特征捕获方面表现中等。

## 建议
1. 对于追求最高预测精度的应用，推荐使用 {comprehensive_results.iloc[0]['Model']}
2. 对于需要高可解释性的临床应用，Cox回归仍是合适选择
3. 随机生存森林可作为两者之间的平衡选择
\"\"\"

with open('../reports/evaluation_report.txt', 'w', encoding='utf-8') as f:
    f.write(report_content)

print("评估报告已保存！")
print("\\n文件保存位置:")
print("- 综合评估结果: ../data/processed/comprehensive_evaluation_results.csv")
print("- 详细评估报告: ../reports/evaluation_report.txt")
print("- 图表文件: ../reports/目录下的所有PNG文件")

print(f"\\n=== 评估总结 ===")
print(f"最佳模型: {comprehensive_results.iloc[0]['Model']}")
print(f"最佳C-index: {comprehensive_results.iloc[0]['C_Index']:.4f}")
print(f"所有模型均具有显著的风险分层能力 (p < 0.05)")
print(f"研究证明了深度学习在癌症生存预测中的优势")